In [1]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql.utils import AnalysisException
from pyspark.ml.feature import RegexTokenizer
from pyspark.sql.functions import udf, mean
from pyspark.sql.types import StringType, StructType, StructField, FloatType
import pandas as pd
import numpy as np
import os, math, time
import itertools
import csv, sys

In [2]:
SAVE = True
file  = open('result.txt', 'w', encoding='utf-8', newline='\n') if SAVE  else sys.stdout

In [3]:
t0 = time.time()

In [4]:
conf = (SparkConf()
        #.setMaster('spark://10.100.5.182:7077')
        #.setMaster("local[4]")
        #.set("spark.executor.memory", "6g")
        .setAppName("quiz"))

In [5]:
try:
    sc = SparkContext(conf=conf)
    sql_sc = SQLContext(sc)
except ValueError:
    pass

In [6]:
def read_csv(file_name):
    try:
        data = sql_sc.read.csv(file_name, 
                       sep=',', 
                       header=False, 
                       mode='DROPMALFORMED')
    except AnalysisException:
        data = sql_sc.read.csv('hdfs:///bdm/quiz/{}'.format(file_name), 
                       sep=',', 
                       header=False, 
                       mode='DROPMALFORMED')
    return data

In [7]:
data = read_csv('kddcup.data')

In [8]:
cloumnnames = ["duration",
               "protocol_type", 
               "service",
               "flag",
               "src_bytes",
               "dst_bytes",
               "land",
               "wrong_fragment",
               "urgent",
               "hot",
               "num_failed_logins",
               "logged_in",
               "num_compromised",
               "root_shell",
               "su_attempted",
               "num_root",
               "num_file_creations",
               "num_shells",
               "num_access_files",
               "num_outbound_cmds",
               "is_host_login",
               "is_guest_login",
               "count",
               "srv_count",
               "serror_rate",
               "srv_serror_rate",
               "rerror_rate",
               "srv_rerror_rate",
               "same_srv_rate",
               "diff_srv_rate",
               "srv_diff_host_rate",
               "dst_host_count",
               "dst_host_srv_count",
               "dst_host_same_srv_rate",
               "dst_host_diff_srv_rate",
               "dst_host_same_src_port_rate",
               "dst_host_srv_diff_host_rate",
               "dst_host_serror_rate",
               "dst_host_srv_serror_rate",
               "dst_host_rerror_rate",
               "dst_host_srv_rerror_rate",
               "intrusion_type"]

In [9]:
for old, new in zip(['_c%d'%(i) for i in range(0, 42)], cloumnnames):
    data = (data.withColumnRenamed(old, new))
data = (data.withColumn('duration', data.duration.cast('float'))
       .withColumn('src_bytes', data.src_bytes.cast('float'))
       .withColumn('dst_bytes', data.dst_bytes.cast('float'))
       .withColumn('num_failed_logins', data.num_failed_logins.cast('float')))

In [10]:
data = data.dropna()

In [11]:
#data = data.sample(False, 0.01, 42)

### (1) For continuous attributes ‘duration’, ‘src_bytes’, ‘dst_bytes’, ‘num_failed_logins’, please calculate their mean, median, mode, standard deviation, respectively

In [12]:
# def find_median(values_list):
#     try:
#         median = np.median(values_list) #get the median of values in a list in each row
#         return round(float(median),2)
#     except Exception:
#         return None #if there is anything wrong with the given values

# median_finder = udf(find_median, FloatType())

In [13]:
q1_feature_nemaes = ['duration', 'src_bytes', 'dst_bytes', 'num_failed_logins']

In [14]:
q1 = data.select('duration', 'src_bytes', 'dst_bytes', 'num_failed_logins')

In [ ]:
print("%s"%(q1.describe().show()), file=file)

In [ ]:
for f in q1_feature_nemaes:
    print('[Q1] %s median:%f'%(f, q1.approxQuantile(f, [0.5], 0.25)[0]), file=file)

In [ ]:
for f in q1_feature_nemaes:
    print('[Q1] %s mode is'%(f), q1.select(f).rdd
    .map(lambda x: (str(x[0]), 1))
    .reduceByKey(lambda a,b:a+b)
    .sortBy(lambda w: w[1], ascending=False)
    .take(1), file=file)

### (2) For symbolic attributes ‘protocol_type’, ‘service’, ‘flag’, ‘logged_in’, ‘intrusion_type’, output the list of each value and the corresponding frequency count, sorted in descending order of the count


In [ ]:
q2_feature_names = ['protocol_type', 'service', 'flag', 'logged_in', 'intrusion_type']

In [ ]:
# (data.select('protocol_type').rdd
# .map(lambda x: (x[0], 1))
# .reduceByKey(lambda a,b:a+b)
# .sortBy(lambda w: w[1], ascending=False)
# .collect())

In [ ]:
for f in q2_feature_names:
    r = (data.select(f).rdd
        .map(lambda x: (x[0], 1))
        .reduceByKey(lambda a,b:a+b)
        .sortBy(lambda w: w[1], ascending=False)
        .collect())
    print('[Q2] column:%s value and the corresponding frequency count'%(f), r, file=file)

### (3) Output the list of the most frequently used ‘service’ for each ‘intrusion_type’, sorted in descending order of the occurrence frequency

In [ ]:
q3_result =  (data.select('service', 'intrusion_type').rdd
.map(lambda x: ((x[0], x[1]), 1))
.reduceByKey(lambda a,b: a+b)
.map(lambda x: (x[0][1], (x[0][0], x[1])))
.groupByKey()
.map(lambda x: (x[0], sorted(x[1], key=lambda x: x[1], reverse=True)[0]))
.collect())
for intrusion_type, r in q3_result:
    print('[Q3]intrusion_type:[%15s],  most frequently used service is:%20s'%(intrusion_type, str(r)),
         file=file)

### (4) If we regard the values of ‘src_bytes’ , calculate the correlation coefficient of src_bytes and ‘num_failed_logins’ by the following formula:


In [ ]:
q4 = data.select('src_bytes', 'num_failed_logins')

In [ ]:
#q4 = (q4.withColumn('is_normal', q4.intrusion_type == 'normal.'))
#q4 = q4.withColumn('is_normal', q4.is_normal.cast('int'))

In [ ]:
corr_coe = q4.collect()

In [ ]:
corr_coe = np.array(corr_coe)

In [ ]:
q4_r = np.corrcoef(corr_coe[:,0], corr_coe[:, 1])

In [ ]:
q4_r[0, 1]

In [ ]:
print('[Q4] correlation coefficient is %f'%(q4_r[0, 1]), file=file) 

### bonus
(5) Which ‘intrusion type’ has the highest value for each of the following fields: 

same_srv_rate

diff_srv_rate

srv_diff_host_rate

dst_host_count

dst_host_srv_count

dst_host_same_srv_rate

dst_host_diff_srv_rate


In [ ]:
#q5 = data.select('intrusion_type', 'same_srv_rate')

In [ ]:
file.close()

In [ ]:
sc.stop()

In [ ]:
print('cost {:.3f} minutes'.format((time.time()-t0)/60))